In [1010]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import functions as f
from datetime import datetime
from functions import FinancialModelingPrepAPI
from functions import fmp

## Cleaning M&A DataFrame from tracxn

Data Source:
https://platform.tracxn.com/a/s/query/t/acquisitions/t/keyacquisitions/table?h=ae589626746705942b5192987ba318555231c920e4537a5b816faa99c9ea5a97&s=sort%3DannouncementDate%7Corder%3DDEFAULT#%7CchartString%3Dview_acquisitionTransactions_acquisitionRoundDate--yearly%2524acquisitiontransactionNormalizedAmount--median--none--none--bar--linear--left--hide%2524numberOfAcquisitions--count--none--none--line--linear--right--hide%7CleaderBoardCompanyType%3DActive

In [1014]:
ma = pd.read_excel('../data/raw/ma-activity-tracxn.xlsx', sheet_name="Acquisitions 1.1")

display(ma.head(15))

/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Acquisitions,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,Details about the acquisitions in your export ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Prepared on Mar 05, 2025 for internal use of r...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Copyright © 2025, Tracxn Technologies Limited....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SNo.,Acquisition Date,Acquisition Type,Acquired Company Name,Acquired Company Domain,Acquirer(s),Sellers,Acquisition Facilitators,Stake Acquired,Acquisition Price (USD),Valuation (USD),Acquisition Multiple(x),is Key Acquisition,Acquired Company Details,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Founded Year,Location,Total Funding (USD),Overview,Business Model
6,1,"Dec 03, 2024",Business Acquisition,NaN,hpspartners.com.sg,BlackRock,NaN,"Perella Weinberg Partners, Morgan Stanley, Cli...",100%,12100000000,NaN,NaN,Yes,NaN,NaN,NaN,Home | HPS Partners | High Performers Search |...,NaN
7,2,"Dec 03, 2024",Business Acquisition,HPS Investment Partners,hpspartners.com,BlackRock,NaN,NaN,NaN,12000000000,NaN,NaN,Yes,2007,"New York City, New York, United States",NaN,Investment management firm investing in public...,Private Debt Funds > Loans and Mezzanine Finan...
8,3,"Jun 30, 2024",Business Acquisition,Preqin,preqin.com,BlackRock,NaN,"Barclays, Skadden, Arps, Slate, Meagher & Flom",NaN,3224245500,NaN,NaN,Yes,2003,"London, England, United Kingdom",NaN,Alternative investments data and insights for ...,Data as a Service > Industries > Investment Da...
9,4,"Jan 12, 2024",Business Acquisition,Global Infrastructure Partners,global-infra.com,BlackRock,NaN,"Fried Frank, Perella Weinberg Partners, Skadde...",100%,12500000000,12500000000,NaN,Yes,2007,"New York City, New York, United States\nLondon...",NaN,Infrastructure and private equity investment f...,"Private Equity Funds > Growth Equity, Infrastr..."


In [1015]:
#Cleaning Headers and rows
ma.columns = [
    "serial_no", "acquisition_date", "acquisition_type", "acquired_company",
    "acquired_company_domain", "acquirer", "sellers", "acquisition_facilitators",
    "stake_acquired", "acquisition_price_usd", "valuation_usd", "acquisition_multiple",
    "is_key_acquisition", "founded_year", "location", "total_funding_usd", "company_details_overview", "business_model"
]
ma.drop(index=ma.index[:7], axis=0, inplace=True)
ma.drop(columns=ma.columns[0], inplace=True)
ma.drop(columns=['sellers','acquisition_facilitators','stake_acquired','valuation_usd','acquisition_multiple','total_funding_usd','business_model'],inplace=True)

In [1018]:
#disclosed acquisition amounts flag

ma['disclosed'] = ma['acquisition_price_usd'].apply(lambda x: 0 if x == 0 else 1)

#Adjusting null values for acquisition prices based on bloomberg and other sources

ma.at[15, 'acquisition_price_usd'] = 1700000000
ma.at[16, 'acquisition_price_usd'] = 0
ma.at[19, 'acquisition_price_usd'] = 700000000
ma.at[22, 'acquisition_price_usd'] = 400000000
ma.at[23, 'acquisition_price_usd'] = 1650000000
ma.at[33, 'acquisition_price_usd'] = 400000000
ma.at[34, 'acquisition_price_usd'] = 0
ma.at[52, 'acquisition_price_usd'] = 35000000
ma.at[82, 'acquisition_price_usd'] = 1500000000

In [1020]:
#adding missing acquisition date
ma['acquisition_date'] = ma['acquisition_date'].fillna(datetime(2002, 1, 1))

In [1022]:
#Fixing datatyes
ma["acquisition_date"] = pd.to_datetime(ma["acquisition_date"], errors="coerce", format='mixed')
ma["acquisition_price_usd"] = (ma["acquisition_price_usd"].astype(float))
ma['founded_year'] = ma['founded_year'].astype(int)
ma['is_key_acquisition'] = ma['is_key_acquisition'].astype(bool)

ma.reset_index(drop=True, inplace=True)

In [1024]:
#Check for null values

ma.isna().sum()

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


acquisition_date            0
acquisition_type            0
acquired_company            0
acquired_company_domain     0
acquirer                    0
acquisition_price_usd       0
is_key_acquisition          0
founded_year                0
location                    0
company_details_overview    0
disclosed                   0
dtype: int64

In [1026]:
ma.dtypes

acquisition_date            datetime64[ns]
acquisition_type                    object
acquired_company                    object
acquired_company_domain             object
acquirer                            object
acquisition_price_usd              float64
is_key_acquisition                    bool
founded_year                         int64
location                            object
company_details_overview            object
disclosed                            int64
dtype: object

In [1028]:
ma['acquirer'].value_counts()

acquirer
Berkshire Hathaway                                                                            27
BlackRock                                                                                     12
Goldman Sachs                                                                                 12
JPMorgan Chase                                                                                 9
State Street                                                                                   3
Onex, Goldman Sachs                                                                            1
MBK Partners, Goldman Sachs                                                                    1
Zoomlion, Hony Capital, Goldman Sachs, Mandarin Capital Partners                               1
Colony Capital, Goldman Sachs                                                                  1
The Carlyle Group, Riverstone Holdings, Goldman Sachs, AIG                                     1
KKR, TPG, Blackstone,

In [1030]:
#adjusting company naming conventions
company_list = ['Berkshire Hathaway', 'BlackRock', 'Goldman Sachs', 'JPMorgan Chase', 'State Street']

company_map = {
    "JP Morgan": "JPMorgan Chase",
    "J P Morgan": "JPMorgan Chase",
    "JP Morgan Chase": "JPMorgan Chase",
    "State St": "State Street",
    "Goldman": "Goldman Sachs",
    "Black Rock": "BlackRock",
    "Berkshire": "Berkshire Hathaway"
}

ma['acquirer'] = ma['acquirer'].apply(lambda x: company_map.get(x.strip(), x) if x.strip() in company_list or x.strip() in company_map else company_list[0])
ma['acquirer'].value_counts()

acquirer
Berkshire Hathaway    55
BlackRock             12
Goldman Sachs         12
JPMorgan Chase         9
State Street           3
Name: count, dtype: int64

In [1032]:
#Industry mapping
company_industries = {
    "ADT": "Security Services",
    "Akaysha Energy": "Energy",
    "Kinder Morgan": "Energy",
    "PCC Energy Group": "Energy",
    "Yongxin Energy": "Energy",
    "Alleghany": "Insurance",
    "Applied Underwriters": "Insurance",
    "Berkshire Hathaway GUARD Insurance": "Insurance",
    "GEICO": "Insurance",
    "Markel Corporation": "Insurance",
    "USI": "Insurance",
    "Aperio": "Investment & Financial Services",
    "Bear Stearns Companies": "Investment & Financial Services",
    "HPS Investment Partners": "Investment & Financial Services",
    "NN Investment Partners": "Investment & Financial Services",
    "Preqin": "Investment & Financial Services",
    "United Capital": "Investment & Financial Services",
    "Washington Mutual": "Investment & Financial Services",
    "iShares": "Investment & Financial Services",
    "Aramark": "Food & Beverage",
    "Hearthside Food Solutions": "Food & Beverage",
    "Heinz": "Food & Beverage",
    "DQ": "Food & Beverage",
    "Burger King": "Food & Beverage",
    "AssuraMed": "Medical & Healthcare",
    "Biomet": "Medical & Healthcare",
    "Capital Vision Services": "Medical & Healthcare",
    "Charles River": "Medical & Healthcare",
    "Icon Cancer Care": "Medical & Healthcare",
    "InstaMed": "Medical & Healthcare",
    "Passport Health Communication": "Medical & Healthcare",
    "Boyd Corporation": "Manufacturing",
    "IMC Group": "Manufacturing",
    "MiTek Industries": "Building & Materials",
    "Scott Fetzer": "Manufacturing",
    "Precision Castparts Corp": "Manufacturing",
    "Sigma Electric": "Manufacturing",
    "BUT": "Retail",
    "Louis": "Retail",
    "DSD": "Retail",
    "Oriental Trading": "Retail",
    "Cachematrix": "Technology & Software",
    "Clarity Money": "Technology & Software",
    "FutureAdvisor": "Technology & Software",
    "Mercatus": "Technology & Software",
    "Nutmeg": "Technology & Software",
    "Solera": "Technology & Software",
    "SunGard": "Technology & Software",
    "eFront": "Technology & Software",
    "Dominion Enterprises": "Media & Entertainment",
    "Endemol Shine Group": "Media & Entertainment",
    "The Infatuation": "Media & Entertainment",
    "Universal Studios Japan": "Media & Entertainment",
    "FlightSafety": "Aerospace & Aviation",
    "Hawker Beechcraft Corporation": "Aerospace & Aviation",
    "NetJets": "Aerospace & Aviation",
    "PCC Aerostructures": "Aerospace Components",
    "Johns Manville": "Building & Materials",
    "Lubrizol": "Building & Materials",
    "Cooper Standard": "Automotive & Transportation",
    "Kenan Advantage Group": "Automotive & Transportation",
    "McClain Trucking Company": "Automotive & Transportation",
    "XTRA Lease": "Automotive & Transportation",
    "CityFibre": "Telecommunications",
    "Unison Site Management": "Telecommunications",
    "Kahoot": "Education",
    "TransUnion": "Banking & Credit Services",
    "Gen Re": "Reinsurance",
    "Neovia Logistics": "Logistics & Fleet Management",
    "LeasePlan": "Logistics & Fleet Management",
    "BNSF Railway": "Railroad Transportation",
    "Bloomspot": "Marketing & Advertising",
    "CIFA": "Construction Equipment",
    "Carver Korea": "Cosmetics & Personal Care",
    "Chore Time Brocks": "Agriculture & Farming Equipment",
    "Clayton Homes": "Housing & Real Estate",
    "Cofense": "Cybersecurity",
    "Currenex": "Financial Services",
    "GreenSky": "Financial Services",
    "Ipreo": "Financial Services",
    "WePay": "Financial Services",
    "SLK": "Financial Services",
    "bear.com": "Financial Services",
    "Global Infrastructure Partners": "Infrastructure Investment",
    "H.H. Brown": "Footwear & Apparel",
    "ISS FS Integrated Solutions": "Facility Services",
    "Nalco": "Chemical Manufacturing",
    "Pilot Flying J": "Travel & Fuel Services",
    "The Marmon Group": "Diversified Industrial",
    "Vastera": "Trade & Supply Chain Management"
}



ma['industry'] = ma['acquired_company'].map(company_industries).fillna('Unknown')

In [1034]:
#Fix datatypes
ma['founded_year'] = ma['founded_year'].astype(int)
ma["acquisition_date"] = pd.to_datetime(ma["acquisition_date"], errors="coerce", format='mixed')
ma['acquisition_year'] = ma['acquisition_date'].dt.year

#Defined matured company if it has been in business for over a decade
ma['matured'] = ma.apply(lambda row: 1 if (row['acquisition_year'] - row['founded_year']) > 10 else 0, axis=1)
ma['matured'] = ma['matured'].astype(bool)

In [1036]:
#added acquired company size and acquisition price in billions
ma['acquisition_price_usd_billions'] = round(ma['acquisition_price_usd'] / 1e9,2)
ma['acquired_company_size'] = ma['acquisition_price_usd_billions'].apply(lambda size: 'Large' if size >= 0.5 
                                                                               else 'Medium' if 0.1 <= size < 0.5
                                                                               else 'Small')


In [1038]:
#Dropping duplicate row
ma.drop(59, axis=0, inplace=True)

In [1040]:
#Dividing 
ma['city'] = ma['location'].apply(lambda x: x.split(',')[0])
ma['state_province_region'] = ma['location'].apply(lambda x: x.split(',')[1])
ma['country'] = ma['location'].apply(lambda x: x.split(',')[-1])
ma.drop(columns='location',inplace=True)
ma

,acquisition_date,acquisition_type,acquired_company,acquired_company_domain,acquirer,acquisition_price_usd,is_key_acquisition,founded_year,company_details_overview,disclosed,industry,acquisition_year,matured,acquisition_price_usd_billions,acquired_company_size,city,state_province_region,country
0,2024-12-03,Business Acquisition,HPS Investment Partners,hpspartners.com,BlackRock,1.200000e+10,True,2007,Investment management firm investing in public...,1,Investment & Financial Services,2024,True,12.00,Large,New York City,New York,United States
1,2024-06-30,Business Acquisition,Preqin,preqin.com,BlackRock,3.224246e+09,True,2003,Alternative investments data and insights for ...,1,Investment & Financial Services,2024,True,3.22,Large,London,England,United Kingdom
2,2024-01-12,Business Acquisition,Global Infrastructure Partners,global-infra.com,BlackRock,1.250000e+10,True,2007,Infrastructure and private equity investment f...,1,Infrastructure Investment,2024,True,12.50,Large,New York City,New York,China
3,2023-07-14,Business Acquisition,Kahoot,kahoot.com,Goldman Sachs,1.700000e+09,True,2012,Cloud-based gamified content management platfo...,1,Education,2023,True,1.70,Large,Oslo,Oslo,Norway
4,2023-03-02,Business Acquisition,Pilot Flying J,pilotflyingj.com,Berkshire Hathaway,1.360000e+10,True,1958,Chain of truck stops located across the US and...,1,Travel & Fuel Services,2023,True,13.60,Large,Knoxville,Tennessee,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,1996-10-15,Business Acquisition,FlightSafety,flightsafety.com,Berkshire Hathaway,1.500000e+09,True,1951,Flight safety training,1,Aerospace & Aviation,1996,True,1.50,Large,New York City,New York,United States
87,1995-08-25,Business Acquisition,GEICO,geico.com,Berkshire Hathaway,2.300000e+09,True,1936,P&C Diversified insurance carrier,1,Insurance,1995,True,2.30,Large,Tucson,Arizona,United States
88,1990-01-01,Business Acquisition,H.H. Brown,hhbrown.com,Berkshire Hathaway,1.610000e+08,True,1883,Manufacturer of footwear,1,Footwear & Apparel,1990,True,0.16,Medium,Greenwich,Connecticut,United States
89,1986-01-01,Business Acquisition,Scott Fetzer,scottfetzer.com,Berkshire Hathaway,3.200000e+08,True,1917,Manufacturer & marketer of equipments for the ...,1,Manufacturing,1986,True,0.32,Medium,Westlake,Ohio,United States


## Cleaning Company Related Data from Financial Modeling Prep API

Data source:

https://site.financialmodelingprep.com/developer/docs

In [742]:
# List of companies and correct tickers
companies = {
    "BlackRock": "BLK",
    "Berkshire Hathaway": "BRK-B",
    "Goldman Sachs": "GS",
    "JPMorgan Chase": "JPM",
    "State Street": "STT"
}

# Fetch Income Statements
blk_income = fmp.get_income_statement(companies["BlackRock"])
brka_income = fmp.get_income_statement(companies["Berkshire Hathaway"])
gs_income = fmp.get_income_statement(companies["Goldman Sachs"])
jpm_income = fmp.get_income_statement(companies["JPMorgan Chase"])
stt_income = fmp.get_income_statement(companies["State Street"])

# Fetch Stock Quotes
blk_stock = fmp.get_stock_quote(companies["BlackRock"])
brka_stock = fmp.get_stock_quote(companies["Berkshire Hathaway"])
gs_stock = fmp.get_stock_quote(companies["Goldman Sachs"])
jpm_stock = fmp.get_stock_quote(companies["JPMorgan Chase"])
stt_stock = fmp.get_stock_quote(companies["State Street"])

# Fetch Company Profiles
blk_profile = fmp.get_company_profile(companies["BlackRock"])
brka_profile = fmp.get_company_profile(companies["Berkshire Hathaway"])
gs_profile = fmp.get_company_profile(companies["Goldman Sachs"])
jpm_profile = fmp.get_company_profile(companies["JPMorgan Chase"])
stt_profile = fmp.get_company_profile(companies["State Street"])

Fetching data from: https://financialmodelingprep.com/api/v3/income-statement/BLK?limit=5
Fetching data from: https://financialmodelingprep.com/api/v3/income-statement/BRK-B?limit=5
Fetching data from: https://financialmodelingprep.com/api/v3/income-statement/GS?limit=5
Fetching data from: https://financialmodelingprep.com/api/v3/income-statement/JPM?limit=5
Fetching data from: https://financialmodelingprep.com/api/v3/income-statement/STT?limit=5
Fetching data from: https://financialmodelingprep.com/api/v3/quote/BLK
Fetching data from: https://financialmodelingprep.com/api/v3/quote/BRK-B
Fetching data from: https://financialmodelingprep.com/api/v3/quote/GS
Fetching data from: https://financialmodelingprep.com/api/v3/quote/JPM
Fetching data from: https://financialmodelingprep.com/api/v3/quote/STT
Fetching data from: https://financialmodelingprep.com/api/v3/profile/BLK
Fetching data from: https://financialmodelingprep.com/api/v3/profile/BRK-B
Fetching data from: https://financialmodeling

In [743]:
# Convert Stock Quotes to DataFrames
blk_stock = f.safe_df_conversion(blk_stock)
brka_stock = f.safe_df_conversion(brka_stock)
gs_stock = f.safe_df_conversion(gs_stock)
jpm_stock = f.safe_df_conversion(jpm_stock)
stt_stock = f.safe_df_conversion(stt_stock)

# Convert Company Profiles to DataFrames
blk_profile = f.safe_df_conversion(blk_profile)
brka_profile = f.safe_df_conversion(brka_profile)
gs_profile = f.safe_df_conversion(gs_profile)
jpm_profile = f.safe_df_conversion(jpm_profile)
stt_profile = f.safe_df_conversion(stt_profile)

In [744]:
#Concat income statements
income_list = [blk_income, brka_income, gs_income, jpm_income, stt_income]
income_df = pd.concat(income_list)

#Concat Stock Quotes
stock_quote_list = [blk_stock, brka_stock, gs_stock, jpm_stock, stt_stock]
stock_df = pd.concat(stock_quote_list)


#Concat Company Profiles
profile_list = [blk_profile, brka_profile, gs_profile, jpm_profile, stt_profile]
profile_df = pd.concat(profile_list)

In [745]:
#Selecting only relevant columns
income_df = income_df[['date', 'symbol', 'revenue', 'grossProfit', 'operatingIncome', 'netIncome', 'eps']]
stock_df = stock_df[['symbol', 'name', 'price', 'changesPercentage', 'marketCap', 'priceAvg50', 'priceAvg200', 'volume', 'eps', 'pe','timestamp']]
profile_df = profile_df[['symbol', 'companyName', 'industry', 'sector', 'mktCap', 'ceo', 'country', 'fullTimeEmployees', 'ipoDate']]

In [746]:
#changing column name
income_df.rename(columns = {'date': 'statement_date'}, inplace=True)
stock_df.rename(columns = {'timestamp': 'stock_timestamp'}, inplace=True)

In [747]:
#Fixing Datatypes
income_df['statement_date'] = pd.to_datetime(income_df['statement_date'])
profile_df['fullTimeEmployees'] = profile_df['fullTimeEmployees'].astype(int)
stock_df['stock_timestamp'] = pd.to_datetime(stock_df['stock_timestamp'])
profile_df['ipoDate'] = pd.to_datetime(profile_df['ipoDate'])

In [748]:
company_map = {
    'BlackRock, Inc.': 'BlackRock',
    'Berkshire Hathaway Inc.': 'Berkshire Hathaway',
    'The Goldman Sachs Group, Inc.': 'Goldman Sachs',
    'JPMorgan Chase & Co.': 'JPMorgan Chase',
    'State Street Corporation': 'State Street'
}


profile_df['companyName'] = profile_df['companyName'].apply(
    lambda x: company_map.get(x.strip(), x.strip()))

stock_df['name'] = stock_df['name'].apply(
    lambda x: company_map.get(x.strip(), x.strip()))

stock_df.rename(columns = {'name':'companyName'}, inplace=True)

In [749]:
#Checking data types
print('Income Statement Data Types')
display(income_df.dtypes)
print("\n")
print('Stock Data Types')
display(stock_df.dtypes)
print("\n")
print('Company Profiles Data Types')
display(profile_df.dtypes)
print("\n")

Income Statement Data Types


statement_date     datetime64[ns]
symbol                     object
revenue                     int64
grossProfit                 int64
operatingIncome             int64
netIncome                   int64
eps                       float64
dtype: object



Stock Data Types


symbol                       object
companyName                  object
price                       float64
changesPercentage           float64
marketCap                     int64
priceAvg50                  float64
priceAvg200                 float64
volume                        int64
eps                         float64
pe                          float64
stock_timestamp      datetime64[ns]
dtype: object



Company Profiles Data Types


symbol                       object
companyName                  object
industry                     object
sector                       object
mktCap                        int64
ceo                          object
country                      object
fullTimeEmployees             int64
ipoDate              datetime64[ns]
dtype: object

## Saving to CSVs & SQL data export

In [1042]:
#save to csv
df_ma = ma

df_ma.to_csv('../data/clean/mergers_acquisitions_data.csv', index=False, sep=';', encoding='utf-8')
income_df.to_csv('../data/clean/income_statement_data.csv', index=False, sep=';', encoding='utf-8')
profile_df.to_csv('../data/clean/profile_data.csv', index=False, sep=';', encoding='utf-8')
stock_df.to_csv('../data/clean/stock_data.csv', index=False, sep=';', encoding='utf-8')

In [1044]:
#Merge DF based on SQL relationships

# Ensure column names are mapped correctly
company_col_map = {'acquirer': 'companyName', 'name': 'companyName'}

# 1. Merging `mergers_acquisitions` with `companies` (profile_df)
merged_ma = df_ma.merge(
    profile_df, 
    left_on='acquirer',
    right_on='companyName',  
    how='left'
)

merged_ma = merged_ma.rename(columns={'industry_x': 'industry'})
merged_ma = merged_ma.drop(columns='industry_y', errors='ignore')
merged_ma = merged_ma.rename(columns={'country_x': 'country'})
merged_ma = merged_ma.drop(columns='country_y', errors='ignore')

merged_ma = merged_ma[['acquisition_date','acquisition_type','acquired_company','acquired_company_domain','acquirer','acquisition_price_usd','is_key_acquisition','founded_year','company_details_overview','disclosed','industry','acquisition_year','matured','city','state_province_region','country']]

# 2. Merging `income_statements` with `companies` (profile_df)
merged_income = income_df.merge(
    profile_df, 
    on='symbol',
    how='left'
)

merged_income = merged_income[['symbol','statement_date','revenue','grossProfit','operatingIncome','netIncome','eps']]

# 3. Merging `stock_prices` with `companies` (profile_df)
merged_stock = stock_df.merge(
    profile_df, 
    left_on='companyName',
    right_on='companyName',  
    how='left'
)


merged_stock = merged_stock.rename(columns={'symbol_x': 'symbol'})
merged_stock = merged_stock.drop(columns='symbol_y', errors
='ignore')

merged_stock = merged_stock[['symbol', 'companyName', 'price', 'changesPercentage', 'marketCap', 'priceAvg50', 'priceAvg200', 'volume', 'eps', 'pe', 'stock_timestamp']]
merged_stock['stock_timestamp'] = merged_stock['stock_timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')


# Save cleaned CSV files for SQL upload
profile_df.to_csv('../sql/data/clean_profiles.csv', index=False, sep=';', encoding='utf-8')
merged_ma.to_csv('../sql/data/clean_mergers_acquisitions.csv', index=False, sep=';', encoding='utf-8')
merged_income.to_csv('../sql/data/clean_income_statements.csv', index=False, sep=';', encoding='utf-8')
merged_stock.to_csv('../sql/data/clean_stock_prices.csv', index=False, sep=';', encoding='utf-8')

In [1046]:
merged_ma


,acquisition_date,acquisition_type,acquired_company,acquired_company_domain,acquirer,acquisition_price_usd,is_key_acquisition,founded_year,company_details_overview,disclosed,industry,acquisition_year,matured,city,state_province_region,country
0,2024-12-03,Business Acquisition,HPS Investment Partners,hpspartners.com,BlackRock,1.200000e+10,True,2007,Investment management firm investing in public...,1,Investment & Financial Services,2024,True,New York City,New York,United States
1,2024-06-30,Business Acquisition,Preqin,preqin.com,BlackRock,3.224246e+09,True,2003,Alternative investments data and insights for ...,1,Investment & Financial Services,2024,True,London,England,United Kingdom
2,2024-01-12,Business Acquisition,Global Infrastructure Partners,global-infra.com,BlackRock,1.250000e+10,True,2007,Infrastructure and private equity investment f...,1,Infrastructure Investment,2024,True,New York City,New York,China
3,2023-07-14,Business Acquisition,Kahoot,kahoot.com,Goldman Sachs,1.700000e+09,True,2012,Cloud-based gamified content management platfo...,1,Education,2023,True,Oslo,Oslo,Norway
4,2023-03-02,Business Acquisition,Pilot Flying J,pilotflyingj.com,Berkshire Hathaway,1.360000e+10,True,1958,Chain of truck stops located across the US and...,1,Travel & Fuel Services,2023,True,Knoxville,Tennessee,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,1996-10-15,Business Acquisition,FlightSafety,flightsafety.com,Berkshire Hathaway,1.500000e+09,True,1951,Flight safety training,1,Aerospace & Aviation,1996,True,New York City,New York,United States
86,1995-08-25,Business Acquisition,GEICO,geico.com,Berkshire Hathaway,2.300000e+09,True,1936,P&C Diversified insurance carrier,1,Insurance,1995,True,Tucson,Arizona,United States
87,1990-01-01,Business Acquisition,H.H. Brown,hhbrown.com,Berkshire Hathaway,1.610000e+08,True,1883,Manufacturer of footwear,1,Footwear & Apparel,1990,True,Greenwich,Connecticut,United States
88,1986-01-01,Business Acquisition,Scott Fetzer,scottfetzer.com,Berkshire Hathaway,3.200000e+08,True,1917,Manufacturer & marketer of equipments for the ...,1,Manufacturing,1986,True,Westlake,Ohio,United States
